In [1]:
from sqlalchemy import create_engine
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv(verbose=True,
            dotenv_path='./.env')
engine = create_engine(f'postgresql://{os.getenv("USERNAME")}:{os.getenv("PASSWORD")}@{os.getenv("HOST")}/{os.getenv("DATABASE")}', 
                       connect_args={'options': '-csearch_path={}'.format('de')})

In [2]:
%%time
concept = pd.read_sql_table('concept', engine)
condition_occurrence = pd.read_sql_table('condition_occurrence', engine)
# takes about 3 min

CPU times: user 34.1 s, sys: 6.16 s, total: 40.2 s
Wall time: 2min 18s


# No.2
환자들이 진단 받은 상병 내역 중 첫글자는 (a,b,c,d,e) 문자로 시작하고 중간에
“heart” 단어가 포함된 상병 이름을 찾으려고 합니다.  
condition_occurrence 테이블은
환자가 병원 방문시 진단 받은 질환이 담겨있습니다.  
상병코드는 condition_concept_id이고, concept 테이블의 concept_id와 조인하여 상병 이름을
찾을 수 있습니다. (concept_name 컬럼 사용)
- 문자 검색시 대소문자를 구분하지 않습니다.
- 상병 이름을 중복없이 나열합니다.
  
  
`condition_occurrence`와 `concept`테이블의 concept_id가 (각각 condition_concept_id, concept_id 사용) 같은 것을 기준으로 join한 후 concept_name에서 조건에 맞는 것들을 찾는다.  
pandas의 merge를 활용해서 join 했고 문자열 찾기는 정규식을 활용했다.
  
  ['chronic congestive heart failure']

In [3]:
condition_occurrence.head()

,condition_occurrence_id,person_id,condition_concept_id,condition_start_date,condition_start_datetime,condition_end_date,condition_end_datetime,condition_type_concept_id,condition_status_concept_id,stop_reason,provider_id,visit_occurrence_id,visit_detail_id,condition_source_value,condition_source_concept_id,condition_status_source_value
0,1466183,116496,0,2002-11-29,2002-11-29,NaT,NaT,32020,0,None,None,36112954,0,162864005,4060985,None
1,1466184,116496,0,2020-03-04,2020-03-04,2020-03-04,2020-03-04,32020,0,None,None,36112952,0,840544004,37311060,None
2,1466185,116496,81151,2015-03-24,2015-03-24,2015-04-14,2015-04-14,32020,0,None,None,7021052,0,44465007,81151,None
3,1466186,116496,260139,2012-10-23,2012-10-23,2012-10-30,2012-10-30,32020,0,None,None,36112948,0,10509002,260139,None
4,1466187,116496,312437,2020-03-04,2020-03-04,2020-04-04,2020-04-04,32020,0,None,None,36112952,0,267036007,312437,None


In [4]:
concept.head()

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,43319838,Juliet-35 Ed (Inert Substance) Sugar Coated Ta...,Drug,AMT,Trade Product Unit,None,87078011000036100,2016-11-01,2099-12-31,None
1,43319839,Junior Cold And Flu Medicine,Drug,AMT,Trade Product,None,65119011000036108,2016-11-01,2099-12-31,None
2,43319840,Laila-35 Ed,Drug,AMT,Trade Product,None,750821000168100,2016-11-01,2099-12-31,None
3,43319841,Laila-35 Ed (Cyproterone Acetate 2 Mg + Ethiny...,Drug,AMT,Trade Product Pack,None,750981000168107,2016-11-01,2099-12-31,None
4,43319842,Laila-35 Ed (Inert Substance) Sugar Coated Tablet,Drug,AMT,Trade Product Unit,None,750841000168106,2016-11-01,2099-12-31,None


In [5]:
# 필요한 열만 남기기
a = condition_occurrence[['condition_concept_id']]
b = concept[['concept_id', 'concept_name']]

In [6]:
# 두 테이블 merge
merged_concept = a.merge(b, 'left', left_on='condition_concept_id', right_on='concept_id')

In [7]:
names = merged_concept['concept_name'].str.lower() # 대소문자 구분하지 않기위해 모두 소문자로

In [ ]:
import re
from collections import defaultdict

answer2 = defaultdict(int)
# a,b,c,d,e로 시작하고 중간에 heart가 들어가는 상병 이름 찾기
comp = re.compile(r'^[abcde].*heart.*')
for n in names:
    if comp.search(n):
        answer2[n] += 1

In [9]:
answer2.keys() # 정답

dict_keys(['chronic congestive heart failure'])

In [10]:
cond = names.str.contains('heart')

In [11]:
names[cond].value_counts() # heart가 들어가는 상병 이름이 몇 개 없다

chronic congestive heart failure    48
heart failure                       11
injury of heart                      5
Name: concept_name, dtype: int64